# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
from scipy.stats import percentileofscore as score
from statistics import mean
from IPython.display import clear_output

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. But, here we have just taken a list of ~500 stocks in a CSV file taken at the time of building the project.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp500.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [3]:
my_columns = ['Ticker',
              'Stock Price',
              'P/E Ratio',
              'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Here to fetch the stock data, we have used `yfinance` API

We can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [4]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    price = data['currentPrice']
    try:
        peRatio = data['currentPrice'] / data['trailingEps']
    except:
        peRatio = np.NaN
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [price],
        'P/E Ratio': [peRatio],
        'Number of Shares to Buy': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(final_dataframe)

    Ticker  Stock Price  P/E Ratio Number of Shares to Buy
0        A       126.30  27.880795                     N/A
1      AAL        15.84   6.187500                     N/A
2      AAP        71.01  10.551263                     N/A
3     AAPL       181.99  30.898132                     N/A
4     ABBV       147.73  34.760000                     N/A
..     ...          ...        ...                     ...
498    YUM       134.35  31.537559                     N/A
499    ZBH       126.32  59.028037                     N/A
500   ZBRA       251.41  32.067602                     N/A
501   ZION        37.92   6.482051                     N/A
502    ZTS       178.63  40.505669                     N/A

[503 rows x 4 columns]


In [5]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,A,126.30,27.880795,N/A
1,AAL,15.84,6.187500,N/A
2,AAP,71.01,10.551263,N/A
3,AAPL,181.99,30.898132,N/A
4,ABBV,147.73,34.760000,N/A
...,...,...,...,...
498,YUM,134.35,31.537559,N/A
499,ZBH,126.32,59.028037,N/A
500,ZBRA,251.41,32.067602,N/A
501,ZION,37.92,6.482051,N/A


In [6]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
80,CAT,276.44,NaN,N/A
378,PODD,263.14,NaN,N/A


Filling the `NULL` cells of the `P/E Ratio` column with mean value

In [7]:
final_dataframe['P/E Ratio'].fillna(final_dataframe['P/E Ratio'].mean(), inplace=True)

In [8]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy


In [9]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,A,126.30,27.880795,N/A
1,AAL,15.84,6.187500,N/A
2,AAP,71.01,10.551263,N/A
3,AAPL,181.99,30.898132,N/A
4,ABBV,147.73,34.760000,N/A
...,...,...,...,...
498,YUM,134.35,31.537559,N/A
499,ZBH,126.32,59.028037,N/A
500,ZBRA,251.41,32.067602,N/A
501,ZION,37.92,6.482051,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll consider only those stocks whose `P/E Ratio` is positive.

In [10]:
final_dataframe.sort_values('P/E Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['P/E Ratio'] > 0]

In [11]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
468,VLO,126.89,3.613041,N/A
384,PSX,110.48,4.220015,N/A
313,MPC,137.95,4.234193,N/A
367,PFG,78.57,4.624485,N/A
166,EQT,41.80,4.644444,N/A
...,...,...,...,...
195,FSLR,194.38,498.410256,N/A
116,CRM,214.59,579.972973,N/A
29,AMD,115.82,643.444444,N/A
286,LVS,57.94,827.714286,N/A


In [12]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [13]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,VLO,126.89,3.613041,N/A
1,PSX,110.48,4.220015,N/A
2,MPC,137.95,4.234193,N/A
3,PFG,78.57,4.624485,N/A
4,EQT,41.80,4.644444,N/A
5,MOS,41.24,5.060123,N/A
6,CTRA,27.57,5.281609,N/A
7,CF,81.24,5.408788,N/A
8,CMA,53.03,5.535491,N/A
9,DVN,50.80,5.539804,N/A


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [14]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [15]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

200000.0

In [16]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,VLO,126.89,3.613041,1576
1,PSX,110.48,4.220015,1810
2,MPC,137.95,4.234193,1449
3,PFG,78.57,4.624485,2545
4,EQT,41.80,4.644444,4784
5,MOS,41.24,5.060123,4849
6,CTRA,27.57,5.281609,7254
7,CF,81.24,5.408788,2461
8,CMA,53.03,5.535491,3771
9,DVN,50.80,5.539804,3937


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

In [17]:
rv_columns = ['Ticker',
              'Stock Price',
              'Number of Shares to Buy',
              'Price-to-Earning Ratio',
              'PE Percentile',
              'Price-to-Book Value',
              'PB Percentile',
              'Price-to-Sales Ratio',
              'PS Percentile',
              'EV/EBITDA',
              'EV/EBITDA Percentile',
              'EV/GP',
              'EV/GP Percentile',
              'RV Score']

rv_dataframe = pd.DataFrame(columns=rv_columns)
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [18]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    financials = yf.Ticker(stock).financials

    try:
        price = data['currentPrice']
    except:
        price = np.NaN

    try:
        eps = data['trailingEps']
    except:
        eps = 1

    try:
        peRatio = price / eps
    except:
        peRatio = np.NaN

    try:
        pbRatio = data['priceToBook']
    except:
        pbRatio = np.NaN

    try:
        marketCap = data['marketCap']
    except:
        marketCap = np.NaN

    try:
        outstandingShares = data['sharesOutstanding']
    except:
        outstandingShares = 1

    try:
        psRatio = marketCap / (outstandingShares * eps)
    except:
        psRatio = np.NaN

    try:
        debt = data['totalDebt']
    except:
        debt = np.NaN

    try:
        cash = data['totalCash']
    except:
        cash = np.NaN

    try:
        ebitda = data['ebitda']
    except:
        ebitda = 1

    try:
        evToEBIDTA = (marketCap + debt - cash) / ebitda
    except:
        evToEBIDTA = np.NaN

    try:
        grossProfit = financials.loc['Gross Profit'][0]
    except:
        grossProfit = 1

    try:
        evToGP = (marketCap + debt - cash) / grossProfit
    except:
        evToGP = np.NaN

    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [price],
        'Number of Shares to Buy': ['N/A'],
        'Price-to-Earning Ratio': [peRatio],
        'PE Percentile': ['N/A'],
        'Price-to-Book Value': [pbRatio],
        'PB Percentile': ['N/A'],
        'Price-to-Sales Ratio': [psRatio],
        'PS Percentile': ['N/A'],
        'EV/EBITDA': [evToEBIDTA],
        'EV/EBITDA Percentile': ['N/A'],
        'EV/GP': [evToGP],
        'EV/GP Percentile': ['N/A'],
        'RV Score': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    rv_dataframe = pd.concat([rv_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(rv_dataframe)

    Ticker  Stock Price Number of Shares to Buy  Price-to-Earning Ratio  \
0        A       126.30                     N/A               27.880795   
1      AAL        15.84                     N/A                6.187500   
2      AAP        71.01                     N/A               10.551263   
3     AAPL       181.99                     N/A               30.898132   
4     ABBV       147.73                     N/A               34.760000   
..     ...          ...                     ...                     ...   
498    YUM       134.35                     N/A               31.537559   
499    ZBH       126.32                     N/A               59.028037   
500   ZBRA       251.41                     N/A               32.067602   
501   ZION        37.92                     N/A                6.482051   
502    ZTS       178.63                     N/A               40.505669   

    PE Percentile  Price-to-Book Value PB Percentile  Price-to-Sales Ratio  \
0             N/A    

In [19]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.30,N/A,27.880795,N/A,6.450789,N/A,27.880796,N/A,1.895191e+01,N/A,1.048415e+01,N/A,N/A
1,AAL,15.84,N/A,6.187500,N/A,NaN,N/A,6.192229,N/A,5.469190e+00,N/A,4.522657e+00,N/A,N/A
2,AAP,71.01,N/A,10.551263,N/A,1.590619,N/A,10.551264,N/A,9.267518e+00,N/A,1.739365e+00,N/A,N/A
3,AAPL,181.99,N/A,30.898132,N/A,46.038452,N/A,30.898133,N/A,2.355809e+01,N/A,1.707562e+01,N/A,N/A
4,ABBV,147.73,N/A,34.760000,N/A,19.631893,N/A,34.759998,N/A,1.115144e+01,N/A,7.784881e+00,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,134.35,N/A,31.537559,N/A,NaN,N/A,31.537559,N/A,2.030849e+01,N/A,1.474469e+01,N/A,N/A
499,ZBH,126.32,N/A,59.028037,N/A,2.131983,N/A,59.028039,N/A,1.354852e+01,N/A,6.497805e+00,N/A,N/A
500,ZBRA,251.41,N/A,32.067602,N/A,4.313386,N/A,32.067603,N/A,1.373726e+01,N/A,5.816346e+00,N/A,N/A
501,ZION,37.92,N/A,6.482051,N/A,1.159952,N/A,6.482051,N/A,8.623620e+09,N/A,8.623620e+09,N/A,N/A


In [20]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
1,AAL,15.84,N/A,6.187500,N/A,NaN,N/A,6.192229e+00,N/A,5.469190,N/A,4.522657,N/A,N/A
19,AIZ,141.78,N/A,31.367257,N/A,1.698635,N/A,3.136726e+01,N/A,NaN,N/A,NaN,N/A,N/A
52,AZO,2450.99,N/A,19.328050,N/A,NaN,N/A,1.932805e+01,N/A,14.253912,N/A,6.452208,N/A,N/A
53,BA,231.36,N/A,-32.631876,N/A,NaN,N/A,-3.263188e+01,N/A,165.293742,N/A,50.739801,N/A,N/A
57,BBWI,37.99,N/A,12.215434,N/A,NaN,N/A,1.221543e+01,N/A,9.037379,N/A,4.186902,N/A,N/A
78,CAH,91.25,N/A,53.676471,N/A,NaN,N/A,5.367647e+01,N/A,10.259414,N/A,3.663293,N/A,N/A
80,CAT,276.44,N/A,276.440000,N/A,NaN,N/A,1.427387e+11,N/A,NaN,N/A,NaN,N/A,N/A
98,CL,76.26,N/A,39.926702,N/A,NaN,N/A,3.992670e+01,N/A,18.021236,N/A,6.944366,N/A,N/A
142,DPZ,395.36,N/A,29.861027,N/A,NaN,N/A,2.986103e+01,N/A,22.281081,N/A,11.539804,N/A,N/A
171,ETSY,81.59,N/A,-14.700901,N/A,NaN,N/A,-1.470090e+01,N/A,26.330018,N/A,6.245606,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data which we have to fill with `mean` values.

In [21]:
for column in ['Price-to-Earning Ratio', 'Price-to-Book Value', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

In [22]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

In [23]:
metrics = {
    'Price-to-Earning Ratio': 'PE Percentile',
    'Price-to-Book Value': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

In [24]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.30,N/A,27.880795,0.622266,6.450789,0.697813,27.880796,0.614314,1.895191e+01,0.664016,1.048415e+01,0.518887,N/A
1,AAL,15.84,N/A,6.187500,0.107356,27.235819,0.923459,6.192229,0.107356,5.469190e+00,0.06163,4.522657e+00,0.145129,N/A
2,AAP,71.01,N/A,10.551263,0.194831,1.590619,0.170974,10.551264,0.194831,9.267518e+00,0.176938,1.739365e+00,0.017893,N/A
3,AAPL,181.99,N/A,30.898132,0.689861,46.038452,0.972167,30.898133,0.677932,2.355809e+01,0.791252,1.707562e+01,0.745527,N/A
4,ABBV,147.73,N/A,34.760000,0.755467,19.631893,0.87674,34.759998,0.741551,1.115144e+01,0.280318,7.784881e+00,0.357853,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,134.35,N/A,31.537559,0.701789,27.235819,0.923459,31.537559,0.691849,2.030849e+01,0.709742,1.474469e+01,0.693837,N/A
499,ZBH,126.32,N/A,59.028037,0.914513,2.131983,0.28827,59.028039,0.900596,1.354852e+01,0.415507,6.497805e+00,0.264414,N/A
500,ZBRA,251.41,N/A,32.067602,0.717694,4.313386,0.554672,32.067603,0.705765,1.373726e+01,0.425447,5.816346e+00,0.2167,N/A
501,ZION,37.92,N/A,6.482051,0.115308,1.159952,0.067594,6.482051,0.115308,8.623620e+09,0.956262,8.623620e+09,0.908549,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [25]:
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

In [26]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.30,N/A,27.880795,0.622266,6.450789,0.697813,27.880796,0.614314,1.895191e+01,0.664016,1.048415e+01,0.518887,0.623459
1,AAL,15.84,N/A,6.187500,0.107356,27.235819,0.923459,6.192229,0.107356,5.469190e+00,0.06163,4.522657e+00,0.145129,0.268986
2,AAP,71.01,N/A,10.551263,0.194831,1.590619,0.170974,10.551264,0.194831,9.267518e+00,0.176938,1.739365e+00,0.017893,0.151093
3,AAPL,181.99,N/A,30.898132,0.689861,46.038452,0.972167,30.898133,0.677932,2.355809e+01,0.791252,1.707562e+01,0.745527,0.775348
4,ABBV,147.73,N/A,34.760000,0.755467,19.631893,0.87674,34.759998,0.741551,1.115144e+01,0.280318,7.784881e+00,0.357853,0.602386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,134.35,N/A,31.537559,0.701789,27.235819,0.923459,31.537559,0.691849,2.030849e+01,0.709742,1.474469e+01,0.693837,0.744135
499,ZBH,126.32,N/A,59.028037,0.914513,2.131983,0.28827,59.028039,0.900596,1.354852e+01,0.415507,6.497805e+00,0.264414,0.55666
500,ZBRA,251.41,N/A,32.067602,0.717694,4.313386,0.554672,32.067603,0.705765,1.373726e+01,0.425447,5.816346e+00,0.2167,0.524056
501,ZION,37.92,N/A,6.482051,0.115308,1.159952,0.067594,6.482051,0.115308,8.623620e+09,0.956262,8.623620e+09,0.908549,0.432604


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [27]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)

In [28]:
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)

In [29]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MHK,105.29,N/A,-48.298165,0.027833,0.817057,0.019881,-48.298167,0.027833,6.968839e+00,0.097416,3.262173e+00,0.071571,0.048907
1,C,45.71,N/A,7.244057,0.133201,0.467034,0.005964,7.244057,0.133201,-3.942013e+11,0.001988,-3.942013e+11,0.001988,0.055268
2,PARA,15.63,N/A,-11.492647,0.05169,0.465664,0.003976,-12.380315,0.049702,8.997549e+00,0.159046,2.466493e+00,0.037773,0.060437
3,EQT,41.80,N/A,4.644444,0.089463,1.259150,0.081511,4.644444,0.089463,2.803803e+00,0.021869,2.305254e+00,0.029821,0.062425
4,T,14.00,N/A,-11.965812,0.049702,0.982249,0.037773,-11.965812,0.05169,6.021491e+00,0.073559,3.708197e+00,0.109344,0.064414
5,MOS,41.24,N/A,5.060123,0.091451,1.104505,0.055666,5.060123,0.091451,4.402008e+00,0.037773,2.900027e+00,0.047714,0.064811
6,WRK,33.89,N/A,-6.916327,0.063618,0.870291,0.025845,-6.916326,0.065606,5.935178e+00,0.067594,4.325726e+00,0.131213,0.070775
7,STT,73.14,N/A,9.897158,0.186879,1.059862,0.047714,9.897158,0.186879,-3.485754e+10,0.00994,-3.485754e+10,0.00994,0.08827
8,VLO,126.89,N/A,3.613041,0.081511,1.733304,0.202783,3.613041,0.081511,2.960741e+00,0.023857,3.111509e+00,0.06163,0.090258
9,VTRS,10.59,N/A,6.745223,0.12326,0.606703,0.007952,6.745223,0.12326,5.815300e+00,0.065606,4.793124e+00,0.157058,0.095427


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [30]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [31]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
position_size

200000.0

In [32]:
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[row, 'Stock Price'])

In [33]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MHK,105.29,1576,-48.298165,0.027833,0.817057,0.019881,-48.298167,0.027833,6.968839e+00,0.097416,3.262173e+00,0.071571,0.048907
1,C,45.71,1810,7.244057,0.133201,0.467034,0.005964,7.244057,0.133201,-3.942013e+11,0.001988,-3.942013e+11,0.001988,0.055268
2,PARA,15.63,1449,-11.492647,0.05169,0.465664,0.003976,-12.380315,0.049702,8.997549e+00,0.159046,2.466493e+00,0.037773,0.060437
3,EQT,41.80,2545,4.644444,0.089463,1.259150,0.081511,4.644444,0.089463,2.803803e+00,0.021869,2.305254e+00,0.029821,0.062425
4,T,14.00,4784,-11.965812,0.049702,0.982249,0.037773,-11.965812,0.05169,6.021491e+00,0.073559,3.708197e+00,0.109344,0.064414
5,MOS,41.24,4849,5.060123,0.091451,1.104505,0.055666,5.060123,0.091451,4.402008e+00,0.037773,2.900027e+00,0.047714,0.064811
6,WRK,33.89,7254,-6.916327,0.063618,0.870291,0.025845,-6.916326,0.065606,5.935178e+00,0.067594,4.325726e+00,0.131213,0.070775
7,STT,73.14,2461,9.897158,0.186879,1.059862,0.047714,9.897158,0.186879,-3.485754e+10,0.00994,-3.485754e+10,0.00994,0.08827
8,VLO,126.89,3771,3.613041,0.081511,1.733304,0.202783,3.613041,0.081511,2.960741e+00,0.023857,3.111509e+00,0.06163,0.090258
9,VTRS,10.59,3937,6.745223,0.12326,0.606703,0.007952,6.745223,0.12326,5.815300e+00,0.065606,4.793124e+00,0.157058,0.095427


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [34]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Formatting Our Excel Output

In [35]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_template = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

float_template = writer.book.add_format(
    {
        'num_format': '0.0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [36]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Stock Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earning Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Value', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

In [37]:
writer.close()